In [49]:
import json
with open('response.json') as f:
    data = json.load(f)
    

In [50]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(data["rows"])
for field in df:
    print(field)
    
df = df[["outputs.query", "inputs.question", "inputs.ground_truth_query", "outputs.sql_similarity.score", "outputs.sql_similarity.explanation"]]

df["better_query"] = None
df["better_query"] = df["better_query"].astype(str)

import os
def save_df(df):
    index = 1
    while True:
        file_name = f"response_with_better_query_{str(index).zfill(3)}.jsonl"
        if not os.path.exists(file_name):
            break
        index += 1
    # save the dataframe to a jsonl file
    df.to_json(file_name, orient="records", lines=True)
    print(f"saved to {file_name}")

outputs.data
outputs.error
outputs.query
outputs.execution_time
inputs.question
inputs.ground_truth_query
outputs.execution_time.seconds
outputs.error.error
outputs.sql_similarity.score
outputs.sql_similarity.explanation
line_number


In [52]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd

# Create widgets
record_number = widgets.HTML()
prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
user_question = widgets.HTML()
output_query = widgets.HTML()
ground_truth_query = widgets.HTML()
sql_similarity_score = widgets.HTML()
sql_similarity_explanation = widgets.HTML()
user_input = widgets.RadioButtons(options=['outputs.query', 'inputs.ground_truth_query', 'None'], description='Better:')
save_button = widgets.Button(description="Save")

# Display widgets
display(widgets.HBox([prev_button, next_button, record_number]))
display(user_question, output_query, ground_truth_query, sql_similarity_score, sql_similarity_explanation, user_input, save_button)

# Initialize index
index = 0

# Function to update widgets
def update_widgets(index):
    user_question.value = f"<div style='word-wrap: break-word;'>{df.loc[index, 'inputs.question']}</div>"
    output_query.value = f"<div style='word-wrap: break-word;'>{df.loc[index, 'outputs.query']}</div>"
    ground_truth_query.value = f"<div style='word-wrap: break-word;'>{df.loc[index, 'inputs.ground_truth_query']}</div>"
    sql_similarity_score.value = f"<div style='word-wrap: break-word;'>{df.loc[index, 'outputs.sql_similarity.score']}</div>"
    sql_similarity_explanation.value = f"<div style='word-wrap: break-word;'>{df.loc[index, 'outputs.sql_similarity.explanation']}</div>"
    user_input.value = df.loc[index, 'better_query'] if 'better_query' in df.columns and pd.notna(df.loc[index, 'better_query']) else None
    record_number.value = f"Record: {index+1}/{len(df)}"

# Function to handle button clicks
def on_next_button_clicked(b):
    global index
    df.loc[index, 'better_query'] = user_input.value
    index = min(index + 1, len(df) - 1)
    update_widgets(index)

def on_prev_button_clicked(b):
    global index
    df.loc[index, 'better_query'] = user_input.value
    index = max(index - 1, 0)
    update_widgets(index)

def on_save_button_clicked(b):
    save_df(df)

# Attach the function to the button
next_button.on_click(on_next_button_clicked)
prev_button.on_click(on_prev_button_clicked)
save_button.on_click(on_save_button_clicked)

# Initialize widgets
update_widgets(index)

HTML(value='')

HTML(value='')

HTML(value='')

HTML(value='')

HTML(value='')

RadioButtons(description='Better:', options=('outputs.query', 'inputs.ground_truth_query', 'None'), value='out…

Button(description='Save', style=ButtonStyle())

saved to response_with_better_query_006.jsonl
saved to response_with_better_query_007.jsonl
saved to response_with_better_query_008.jsonl
saved to response_with_better_query_009.jsonl


In [53]:
#show rows of df where better_query is not 'outputs.query'
df[df["better_query"] != "outputs.query"]
# set fiel ground_truth_query to whaterver is better_query says (data.ground_truth_query or outputs.query)
df["ground_truth_query"] = None
# set question to data.question
df["question"] = df["inputs.question"]
for index, row in df.iterrows():
    if row["better_query"] == "outputs.query":
        df.at[index, "ground_truth_query"] = row["outputs.query"]
    else:
        df.at[index, "ground_truth_query"] = row["inputs.ground_truth_query"]




In [55]:
save_df(df[["question", "ground_truth_query"]])

saved to response_with_better_query_001.jsonl
